In [ ]:
!pip install datasets
!pip install transformers
!pip install torch torchaudio

from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
from datasets import load_dataset, Audio
# from IPython.display import Audio
import torch
import torchaudio
import pickle
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/5541/'
import pandas as pd

In [ ]:
pipe = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [ ]:
path = '/content/drive/MyDrive/5541/dialect_to_standard_viet'

In [ ]:
# dataset = load_dataset("nguyendv02/ViMD_Dataset", split="train", streaming=True)
# subset = [next(iter(dataset)) for _ in range(1)]
# subset[0]
# with open(path+"/data_point_sample.pkl", "wb") as f:
#     pickle.dump(subset[0], f)
with open(path+"/data_point_sample.pkl", "rb") as f:
    data_point = pickle.load(f)

In [ ]:
# Extract audio array and original sampling rate
original_waveform = np.array(data_point["audio"]["array"])
original_waveform = torch.from_numpy(original_waveform).float()

original_sampling_rate = data_point["audio"]["sampling_rate"]

# Define resampling transform (from 44.1 kHz to 16 kHz)
resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=16000)

# Apply resampling
resampled_waveform = resampler(original_waveform)

# Update the data point with the resampled audio
data_point["audio"]["array"] = resampled_waveform.numpy()  # Convert back to NumPy array
data_point["audio"]["sampling_rate"] = 16000


In [ ]:
# processor = AutoProcessor.from_pretrained("vinai/PhoWhisper-large")
# model = AutoModelForSpeechSeq2Seq.from_pretrained("vinai/PhoWhisper-large")
pipe = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-large")

Device set to use cuda:0


In [ ]:
audio_array = data_point["audio"]["array"]  # NumPy array
sampling_rate = data_point["audio"]["sampling_rate"]  # Should be 16000
transcription = pipe({"array": audio_array, "sampling_rate": sampling_rate})
transcription

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': 'nghiên cứu học tập các ứng dụng các khoa công nghệ và những ứng dụng tiên tiến để đưa vào trong áp dụng trong công việc của mình từ đó đức chuyển đổi những cái khó khăn trong công việc đưa ra những bận lợi và những cái nhanh gọn hơn giúp giải quyết các trục tục bình tĩnh giải quyết các trục tục bình chính để họ được thuận tiện hơn.'}

In [ ]:
data_point['text']

'Nghiên cứu học tập,các ứng dụng các khoa học công nghệ và những dụng tiên tiến để đưa vào trong áp dụng trong công việc của mình. Từ đó giúp chuyển đổi những cái khó khăn trong công việc, đưa ra những thuận lợi và những cái nhanh gọn hơn giải quyết các thủ tục hành chính cho được thuận tiện hơn.'

## phowhisper for nghe an

In [ ]:

df = pd.read_parquet(path+"nghe_an.parquet")
print(df.head())


    region  province_code province_name     filename  \
0  Central             37        NgheAn  37_0001.wav   
1  Central             37        NgheAn  37_0002.wav   
2  Central             37        NgheAn  37_0003.wav   
3  Central             37        NgheAn  37_0004.wav   
4  Central             37        NgheAn  37_0005.wav   

                                                text    speakerID  gender  \
0  Trẻ em thành phố hiện nay ngoài học thêm và ở ...  spk_37_0001       0   
1  Dịp này cháu nghỉ hè, phải cho cháu về quê. Cò...  spk_37_0002       0   
2  Chơi dưới đường thì cũng không được an toàn bở...  spk_37_0003       1   
3  Trên địa bàn thị xã Cửa Lò thiếu những cái điể...  spk_37_0004       1   
4  Về cái quy hoạch, tổng thể trước đây thì không...  spk_37_0005       1   

                                               audio  
0  {'array': [9.1552734375e-05, 6.103515625e-05, ...  
1  {'array': [0.00048828125, 0.00067138671875, 0....  
2  {'array': [0.000396728515625, 0.

In [ ]:
print(df.shape)

(280, 8)


In [ ]:
print(df['audio'][0])

{'array': array([9.15527344e-05, 6.10351562e-05, 0.00000000e+00, ...,
       1.73645020e-02, 2.27050781e-02, 1.90429688e-02]), 'path': '37_0001.wav', 'sampling_rate': 44100}


In [ ]:

inputs = []

for i in range(df.shape[0]):

  original_waveform = np.array(df["audio"][i]["array"])
  original_waveform = torch.from_numpy(original_waveform).float()

  original_sampling_rate = df["audio"][i]["sampling_rate"]

  # Define resampling transform (from 44.1 kHz to 16 kHz)
  resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=16000)

  # Apply resampling
  resampled_waveform = resampler(original_waveform)

  # Update the data point with the resampled audio
  df["audio"][i]["array"] = resampled_waveform.numpy()  # Convert back to NumPy array
  df["audio"][i]["sampling_rate"] = 16000

  inputs.append({'array': df['audio'][i]['array'], 'sampling_rate': df['audio'][i]['sampling_rate']})



# audio_array = df["audio"][i]["array"]  # NumPy array
# sampling_rate = df["audio"][i]["sampling_rate"]  # Should be 16000



In [ ]:
transcription = pipe(inputs, return_timestamps=True)

transcription

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can h

[{'text': 'trẻ em thành phố hiện nay ngoài học thêm và ở nhà ráy thì chưa có một sân trường dành riêng cho các trẻ em ngoài ở công viên và hồ điều hoà dành cho cả người lớn và trẻ nhỏ thì ở trung tâm thương mại thì các trẻ em lại phải mất tiền công viên và hồ điều hoà thì lại chưa có sự kiểm soát an toàn đối với các trẻ nhỏ nên là em mong muốn có một khu vui chơi sân vui chơi dành cho trẻ nhỏ được kiểm soát an toàn và lành mạnh.',
  'chunks': [{'timestamp': (0.0, 29.58),
    'text': 'trẻ em thành phố hiện nay ngoài học thêm và ở nhà ráy thì chưa có một sân trường dành riêng cho các trẻ em ngoài ở công viên và hồ điều hoà dành cho cả người lớn và trẻ nhỏ thì ở trung tâm thương mại thì các trẻ em lại phải mất tiền công viên và hồ điều hoà thì lại chưa có sự kiểm soát an toàn đối với các trẻ nhỏ nên là em mong muốn có một khu vui chơi sân vui chơi dành cho trẻ nhỏ được kiểm soát an toàn và lành mạnh.'}]},
 {'text': 'dịp này cháu nghỉ hè phải cho cháu về quê còn ở vinh là cứ nhốt cháu vào 

In [ ]:
print(len(transcription))

280


In [ ]:
print(df['text'][0])

Trẻ em thành phố hiện nay ngoài học thêm và ở nhà ra thì chưa có một sân chơi dành riêng cho các trẻ em. Ngoài ở công viên và hồ Điều Hòa là dành cho cả người lớn và trẻ nhỏ thì ở trung tâm thương mại thì các trẻ em lại phải mất tiền. Công viên và hồ Điều Hòa thì Lại chưa có sự kiểm soát sự an toàn đối với các trẻ nhỏ. Nên là em mong muốn có một khu vui chơi, sân vui chơi dành cho trẻ nhỏ được kiểm soát an toàn và lành mạnh.


In [ ]:
df['pho_transcription'] = transcription

In [ ]:
df.to_parquet(path+"nghe_an_transcription.parquet")

In [ ]:
df.to_csv(path+"nghe_an_transcription.csv")

## transcription for Ha Noi

In [ ]:
df_hn = pd.read_parquet(path+"hanoi.parquet")
df_hn.head()

,region,province_code,province_name,filename,text,speakerID,gender,audio
0,North,30,HaNoi,30_0001.wav,Mình tên là Phạm Thị Thu Hoài. Hiện tại thì mì...,spk_30_0001,0,"{'array': [0.0, 1.52587890625e-05, 4.577636718..."
1,North,30,HaNoi,30_0002.wav,Thời gian đầu mà sử dụng app hẹn hò tôi không ...,spk_30_0001,0,"{'array': [0.006195068359375, 0.006591796875, ..."
2,North,30,HaNoi,30_0003.wav,Cuộc sống hàng ngày của mình chỉ đi làm tới cô...,spk_30_0001,0,"{'array': [0.002349853515625, 0.00186157226562..."
3,North,30,HaNoi,30_0004.wav,Mình thấy môi trường trên app hẹn Hò khá là th...,spk_30_0001,0,"{'array': [0.010345458984375, 0.010009765625, ..."
4,North,30,HaNoi,30_0005.wav,Mình có gặp và được trò chuyện với một anh ngư...,spk_30_0001,0,"{'array': [0.00079345703125, 0.001068115234375..."


In [ ]:
hanoi = []

for i in range(df_hn.shape[0]):

  original_waveform = np.array(df_hn["audio"][i]["array"])
  original_waveform = torch.from_numpy(original_waveform).float()

  original_sampling_rate = df_hn["audio"][i]["sampling_rate"]

  # Define resampling transform (from 44.1 kHz to 16 kHz)
  resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=16000)

  # Apply resampling
  resampled_waveform = resampler(original_waveform)

  # Update the data point with the resampled audio
  df_hn["audio"][i]["array"] = resampled_waveform.numpy()  # Convert back to NumPy array
  df_hn["audio"][i]["sampling_rate"] = 16000

  hanoi.append({'array': df_hn['audio'][i]['array'], 'sampling_rate': df_hn['audio'][i]['sampling_rate']})

In [ ]:
transcription = pipe(hanoi, return_timestamps=True)

transcription

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can hap

[{'text': 'mình tên là phạm thị thu hoài hiện tại thì mình đang ở quận cầu giấy.',
  'chunks': [{'timestamp': (0.02, 4.14),
    'text': 'mình tên là phạm thị thu hoài hiện tại thì mình đang ở quận cầu giấy.'}]},
 {'text': 'thời gian đầu mà sử dụng tờ áp hẹn hò tớ không chủ đích lắm nghĩa là những cái áp đầu tiên là có phây búc cái phần hẹn hò trên phây búc ạ thì thấy là có những cái gương mặt rất lạ mình chưa gặp bao giờ và thấy cũng điều rất hay là họ sẵn sàng để những cái thông tin trên bai ô của họ ấy thì đó là cái điểm chạm đầu tiên mà tớ tiếp cận với áp hẹn hò sau đó thì tớ có thử sử dụng tinder và trong khoảng một năm gần đây thì mình có chuyển sang dùng băm bồ.',
  'chunks': [{'timestamp': (0.0, None),
    'text': 'thời gian đầu mà sử dụng tờ áp hẹn hò tớ không chủ đích lắm nghĩa là những cái áp đầu tiên là có phây búc cái phần hẹn hò trên phây búc ạ thì thấy là có những cái gương mặt rất lạ mình chưa gặp bao giờ và thấy cũng điều rất hay là họ sẵn sàng để những cái thông tin tr

In [ ]:
print(len(transcription))

248


In [ ]:
print(len(df_hn))

248


In [ ]:
df_hn['pho_transcription'] = transcription

In [ ]:
df_hn.to_parquet(path+"hanoi_transcription.parquet")
df_hn.to_csv(path+"hanoi_transcription.csv")

## transcription for Sai Gon

In [ ]:
df_sg = pd.read_parquet(path+"saigon.parquet")
df_sg.head()

,region,province_code,province_name,filename,text,speakerID,gender,audio
0,South,59,HoChiMinh,59_0223.wav,"Mặt hàng thì cũng không có thiếu thốn, siêu th...",spk_59_0191,0,"{'array': [-0.005645751953125, -0.002685546875..."
1,South,59,HoChiMinh,59_0224.wav,Cái giá này thì thấy cũng tạm ổn được chút xíu...,spk_59_0192,0,"{'array': [-0.01214599609375, -0.0128479003906..."
2,South,59,HoChiMinh,59_0225.wav,"Đơn hàng cũng trên mười nghìn đơn, mười nghìn ...",spk_59_0193,1,"{'array': [0.0, 1.52587890625e-05, 0.0, -3.051..."
3,South,59,HoChiMinh,59_0226.wav,sỞ công thương phối hợp để mở rộng vùng cung ứ...,spk_59_0194,1,"{'array': [-0.0001220703125, -0.0001220703125,..."
4,South,59,HoChiMinh,59_0227.wav,Đối với tôi thì vắc xin nào cũng không phải qu...,spk_59_0195,1,"{'array': [1.52587890625e-05, 0.0, -3.05175781..."


In [ ]:
saigon = []

for i in range(df_sg.shape[0]):

  original_waveform = np.array(df_sg["audio"][i]["array"])
  original_waveform = torch.from_numpy(original_waveform).float()

  original_sampling_rate = df_sg["audio"][i]["sampling_rate"]

  # Define resampling transform (from 44.1 kHz to 16 kHz)
  resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=16000)

  # Apply resampling
  resampled_waveform = resampler(original_waveform)

  # Update the data point with the resampled audio
  df_sg["audio"][i]["array"] = resampled_waveform.numpy()  # Convert back to NumPy array
  df_sg["audio"][i]["sampling_rate"] = 16000

  saigon.append({'array': df_sg['audio'][i]['array'], 'sampling_rate': df_sg['audio'][i]['sampling_rate']})

In [ ]:
transcription = pipe(saigon, return_timestamps=True)

transcription

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can h

[{'text': 'mặt hàng thì cũng không có thiếu thốn siêu thị thì cũng bán bình thường.',
  'chunks': [{'timestamp': (0.02, 4.1),
    'text': 'mặt hàng thì cũng không có thiếu thốn siêu thị thì cũng bán bình thường.'}]},
 {'text': 'cái giá này thì thấy cũng tạm ổn được chút xíu đỡ hơn mọi bữa.',
  'chunks': [{'timestamp': (0.02, 4.74),
    'text': 'cái giá này thì thấy cũng tạm ổn được chút xíu đỡ hơn mọi bữa.'}]},
 {'text': 'đơn hàng cũng trên mười nghìn đơn nghĩa là mười nghìn hộ dân đã đã có nhu cầu và mà bọn tôi cũng đã giao được tương đối đầy đủ đến thời điểm hiện tại là cũng được đến tám mươi phần trăm đây thì là cái cái đấy là cái tương đối tích cực khi mà người dân không phải di chuyển ra ngoài.',
  'chunks': [{'timestamp': (0.02, None),
    'text': 'đơn hàng cũng trên mười nghìn đơn nghĩa là mười nghìn hộ dân đã đã có nhu cầu và mà bọn tôi cũng đã giao được tương đối đầy đủ đến thời điểm hiện tại là cũng được đến tám mươi phần trăm đây thì là cái cái đấy là cái tương đối tích cực 

In [ ]:
df_sg['pho_transcription'] = transcription

In [ ]:
df_sg.to_parquet(path+"saigon_transcription.parquet")
df_sg.to_csv(path+"saigon_transcription.csv")

## transcription for hue

In [ ]:
df_hue = pd.read_parquet(path+"hue.parquet")
df_hue.head()

,region,province_code,province_name,filename,text,speakerID,gender,audio
0,Central,75,ThuaThienHue,75_0001.wav,"Cái từ tiếp theo, cái từ cảm thán tiếp theo mà...",spk_75_0001,0,"{'array': [0.002685546875, 0.0019073486328125,..."
1,Central,75,ThuaThienHue,75_0002.wav,"Kiểu giỏi vô hậu luôn á, anh giỏi vô hậu luôn ...",spk_75_0001,0,"{'array': [-0.0225830078125, -0.00997924804687..."
2,Central,75,ThuaThienHue,75_0003.wav,Dạo gần đây thì có rất nhiều bạn nhắn tin cho ...,spk_75_0002,0,"{'array': [0.009857177734375, 0.01449584960937..."
3,Central,75,ThuaThienHue,75_0004.wav,Có những ngày mình cực kỳ nhớ Huế. Khi mà mình...,spk_75_0002,0,"{'array': [-0.057525634765625, -0.053131103515..."
4,Central,75,ThuaThienHue,75_0005.wav,Dù vậy thì người đến Huế một lần và người đã s...,spk_75_0002,0,"{'array': [-0.027679443359375, -0.027481079101..."


In [ ]:
hue = []

for i in range(df_hue.shape[0]):

  original_waveform = np.array(df_hue["audio"][i]["array"])
  original_waveform = torch.from_numpy(original_waveform).float()

  original_sampling_rate = df_hue["audio"][i]["sampling_rate"]

  # Define resampling transform (from 44.1 kHz to 16 kHz)
  resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=16000)

  # Apply resampling
  resampled_waveform = resampler(original_waveform)

  # Update the data point with the resampled audio
  df_hue["audio"][i]["array"] = resampled_waveform.numpy()  # Convert back to NumPy array
  df_hue["audio"][i]["sampling_rate"] = 16000

  hue.append({'array': df_hue['audio'][i]['array'], 'sampling_rate': df_hue['audio'][i]['sampling_rate']})

In [ ]:
transcription = pipe(hue, return_timestamps=True)

transcription

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can h

[{'text': 'cái từ tiếp theo cái từ cảm thông tiếp theo mà mình muốn nhắc đến cho các bạn đó là cái từ cảm thông mình nghĩ là nó có mức độ cao nhất trong tất cả những cái từ mà mình đưa ra thì ngoài cái từ dễ sợ dã man thì một cái từ có mức độ cao hơn nữa đó là cái từ vô hậu có nghĩa là kiểu ở đây là có ý tích cực thôi chứ không phải là có ý tiêu cực kỳ đâu các bạn nó chỉ nhấn mạnh thêm thôi.',
  'chunks': [{'timestamp': (0.0, None),
    'text': 'cái từ tiếp theo cái từ cảm thông tiếp theo mà mình muốn nhắc đến cho các bạn đó là cái từ cảm thông mình nghĩ là nó có mức độ cao nhất trong tất cả những cái từ mà mình đưa ra thì ngoài cái từ dễ sợ dã man thì một cái từ có mức độ cao hơn nữa đó là cái từ vô hậu có nghĩa là kiểu ở đây là có ý tích cực thôi chứ không phải là có ý tiêu cực kỳ đâu các bạn nó chỉ nhấn mạnh thêm thôi.'}]},
 {'text': 'kiểu mình giỏi vô hầu luôn á anh anh giỏi vô hầu luôn á trời ơi muốn mồ dí sờ luôn á kiểu như vậy hoặc là mình ăn món gì đó mới ngon quá ngon quá mà m

In [ ]:
df_hue['pho_transcription'] = transcription

In [ ]:
df_hue.to_parquet(path+"hue_transcription.parquet")
df_hue.to_csv(path+"hue_transcription.csv")